# Week 4: Scale-Free Networks, Hubs & Resilience

**Learning objectives** — After this lab you should be able to:

- Explain why ER graphs cannot produce hubs (recap from Week 3)
- Describe preferential attachment and its role in hub formation
- Compare degree distributions of synthetic models to real networks
- Visualize power laws using CCDF and estimate exponents with MLE
- Explain the Molloy-Reed criterion for giant component existence
- Demonstrate the robustness paradox: robust to random failure, fragile to targeted attack

Why do some airports have 200 routes while most have only 5? Why do a few Twitter accounts
have millions of followers? Last week we saw that real networks are small worlds and explored
the ER random model as a baseline. This week we discover that only one model can explain the
extreme **hubs** we see in real data — and we'll see why those hubs make networks both
powerful and vulnerable.

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import poisson
from netsci.loaders import load_graph
from netsci.utils import SEED, graph_summary, fit_power_law
from netsci import viz, models

---
## 1. Datasets

We revisit three familiar networks this week:

- **Facebook** (334 nodes) — a dense social graph with a fat-tailed degree distribution and high clustering. Its hubs and tight communities make it a good test case for the BA model.
- **US Power Grid** (4,941 nodes) — a sparse infrastructure network with narrow degree distribution and low clustering. Its engineering constraints make it a challenge for all models.
- **US Airports** (500 nodes, ~2,980 edges) — a hub-and-spoke topology where a few major airports connect to hundreds of others. We’ll use this later to study how hubs affect network resilience.

In [ ]:
G_fb = load_graph("facebook")
graph_summary(G_fb)
print()
G_pg = load_graph("powergrid")
graph_summary(G_pg)
print()
G_air = load_graph("airports")
graph_summary(G_air)

---
## 2. The Hub Puzzle

Let's look at the Facebook degree distribution on a log-log scale:

In [ ]:
viz.plot_degree_dist(G_fb, log=True, title="Facebook — degree distribution (log-log)")

The **fat tail** on the right shows that a few nodes have enormous degree — these are **hubs**.
Most nodes have low degree, but the hubs dominate the network's structure.

Can our models explain this? Let's find out.

---
## 3. ER & WS Recap

Last week we explored the **Erdos-Renyi** and **Watts-Strogatz** models in detail.
Let's quickly generate both and ask: can either explain the hubs we see in Facebook?

In [ ]:
# Quick recap: generate ER and WS models, compare to Facebook
G_er = models.erdos_renyi(n=500, avg_degree=6)
G_ws = models.watts_strogatz(n=500, k=6, p=0.1)

fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))
for ax, (name, G) in zip(
    axes, [("Erdos-Renyi", G_er), ("Watts-Strogatz", G_ws), ("Facebook", G_fb)]
):
    degrees = [d for _, d in G.degree()]
    deg_count = Counter(degrees)
    x, y = zip(*sorted(deg_count.items()))
    ax.scatter(x, y, s=30, alpha=0.7, edgecolors="white", linewidth=0.5)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Degree (k)")
    ax.set_ylabel("Count")
    ax.set_title(f"{name} (log-log)")
fig.suptitle("Can ER or WS explain hubs?", fontsize=13, fontweight="bold")
fig.tight_layout()
plt.show()

# Stats comparison
print(f"{'Model':20s} {'Max degree':>10s} {'Clustering':>10s}")
print("-" * 42)
for name, G in [("ER (n=500)", G_er), ("WS (n=500)", G_ws), ("Facebook", G_fb)]:
    max_d = max(d for _, d in G.degree())
    C = nx.average_clustering(G)
    print(f"{name:20s} {max_d:10d} {C:10.4f}")

**Neither model explains hubs.** ER produces a narrow Poisson degree distribution — no fat tail. WS starts from a regular lattice and rewiring barely changes the degree distribution. Both models produce max degrees around 10-15, while Facebook has nodes with 100+ connections. We need a fundamentally different mechanism.

---
## 4. Barabasi-Albert Model

The key idea is **preferential attachment** — "the rich get richer."

New nodes join the network one at a time. Each new node connects to *m* existing nodes,
but prefers to connect to nodes that **already have many connections**.
This naturally produces a **power-law** degree distribution with hubs.

The BA model rests on two simple rules:

1. **Growth** — the network starts small and new nodes arrive one at a time (unlike ER/WS which start with all nodes present).
2. **Preferential attachment** — each new node connects to *m* existing nodes, but prefers high-degree nodes. The probability of connecting to node *i* is proportional to its current degree: P(i) = k_i / Σk.

The result: early nodes accumulate connections over time (“the rich get richer”), producing the extreme hubs we see in real networks.

In [ ]:
G_ba = models.barabasi_albert(n=500, m=3)
graph_summary(G_ba)
print(f"Max degree: {max(d for _, d in G_ba.degree())}")

In [ ]:
viz.plot_degree_dist(G_ba, log=True, title="Barabasi-Albert (log-log) — power law!")

**The straight line**: On a log-log scale, the BA degree distribution approximates a straight line — the hallmark of a **power law** P(k) ~ k^(-γ) with γ ≈ 3. The maximum degree (typically 30-80 for n=500) is far larger than ER’s maximum (~15-20). These extreme hubs emerge naturally from preferential attachment without any explicit “hub creation” rule.

**Try it yourself**: Generate two BA graphs with m=1 and m=5 (both n=500). Which has a higher max degree? Why does this make sense given the preferential attachment mechanism?

In [ ]:
# YOUR CODE HERE
G_ba_m1 = models.barabasi_albert(n=500, m=1)
G_ba_m5 = models.barabasi_albert(n=500, m=5)

max_deg_m1 = max(d for _, d in G_ba_m1.degree())
max_deg_m5 = max(d for _, d in G_ba_m5.degree())
assert max_deg_m5 > max_deg_m1, (
    "Hint: m=5 should produce a higher max degree — each new node adds more edges"
)
print(
    f"BA(m=1): max degree = {max_deg_m1}, avg degree = {2 * G_ba_m1.number_of_edges() / 500:.1f}"
)
print(
    f"BA(m=5): max degree = {max_deg_m5}, avg degree = {2 * G_ba_m5.number_of_edges() / 500:.1f}"
)
print(f"m=5 produces larger hubs because more edges feed the rich-get-richer process")

On the log-log plot, BA degree distribution approximates a straight line — the hallmark of a **power law**. Hubs emerge naturally from the preferential attachment process.

In [ ]:
# BA growth snapshots: same graph at n=5, n=10, n=20
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))
snapshots = [5, 10, 20]

# Build a BA graph incrementally up to max snapshot size
G_grow = nx.barabasi_albert_graph(max(snapshots), 2, seed=SEED)

for ax, n_snap in zip(axes, snapshots):
    G_sub = G_grow.subgraph(range(n_snap)).copy()
    pos = nx.spring_layout(G_sub, seed=SEED)
    degrees = dict(G_sub.degree())
    sizes = [degrees[n] * 80 + 40 for n in G_sub.nodes()]
    nx.draw_networkx(
        G_sub,
        pos,
        ax=ax,
        node_color="#4878CF",
        node_size=sizes,
        edge_color="#cccccc",
        width=1.0,
        with_labels=True,
        font_size=8,
        font_color="white",
    )
    ax.set_title(f"n = {n_snap}\nmax degree = {max(degrees.values())}", fontsize=11)
    ax.axis("off")

fig.suptitle("BA Growth: node size ∙ degree (early nodes become hubs)", fontsize=13)
fig.tight_layout()
plt.show()

---
## 5. Side-by-Side Comparison

This is the centerpiece figure: three models compared across degree distribution, graph structure, and statistics.

**Before you look**: Based on what you know about each model, predict:
- Which model will show a **straight line** on the log-log degree plot? (Hint: power law)
- Which will have the **highest clustering**? (Hint: which starts from a lattice?)
- Which will have the **largest max degree**? (Hint: rich-get-richer)

Check your predictions against the 3×3 panel below.

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(16, 14))

model_graphs = [
    ("Erdos-Renyi", G_er),
    ("Watts-Strogatz", G_ws),
    ("Barabasi-Albert", G_ba),
]

for row, (name, G) in enumerate(model_graphs):
    # Column 0: Degree distribution (log-log)
    ax = axes[row, 0]
    degrees = [d for _, d in G.degree()]
    deg_count = Counter(degrees)
    x, y = zip(*sorted(deg_count.items()))
    ax.scatter(x, y, s=20, alpha=0.7, edgecolors="white", linewidth=0.3)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Degree (k)")
    ax.set_ylabel("Count")
    ax.set_title(f"{name}\nDegree dist (log-log)")

    # Column 1: Graph drawing (subset of 100 nodes for readability)
    ax = axes[row, 1]
    nodes_sub = list(G.nodes())[:100]
    G_sub = G.subgraph(nodes_sub)
    pos = nx.spring_layout(G_sub, seed=SEED)
    nx.draw_networkx(
        G_sub,
        pos,
        ax=ax,
        node_color="#4878CF",
        node_size=20,
        edge_color="#cccccc",
        width=0.3,
        with_labels=False,
        alpha=0.8,
    )
    ax.set_title(f"{name}\nGraph (first 100 nodes)")
    ax.axis("off")

    # Column 2: Statistics
    ax = axes[row, 2]
    ax.axis("off")
    stats_text = (
        f"Nodes: {G.number_of_nodes()}\n"
        f"Edges: {G.number_of_edges()}\n"
        f"Avg degree: {np.mean(degrees):.1f}\n"
        f"Max degree: {max(degrees)}\n"
        f"Clustering: {nx.average_clustering(G):.4f}\n"
    )
    ax.text(
        0.1,
        0.5,
        stats_text,
        transform=ax.transAxes,
        fontsize=13,
        verticalalignment="center",
        fontfamily="monospace",
    )
    ax.set_title(f"{name}\nStatistics")

fig.suptitle("Three Network Models Compared", fontsize=16, y=1.01)
fig.tight_layout()
plt.show()

**Reading the statistics column**: ER has low clustering (~0.02) and moderate max degree. WS has the highest clustering (~0.4) but a narrow max degree. BA has the highest max degree but low clustering (~0.01). No single model captures both hubs *and* clustering simultaneously — this is a fundamental limitation that drives research into more advanced models.

---
## 6. Model vs Reality

How well does BA match the Facebook network?

In [ ]:
# Generate a BA graph with same n as Facebook
n_fb = G_fb.number_of_nodes()
avg_deg_fb = 2 * G_fb.number_of_edges() / n_fb
m_ba = max(1, round(avg_deg_fb / 2))  # BA avg_deg ≈ 2m
G_ba_fb = models.barabasi_albert(n_fb, m_ba)

# Overlay degree distributions
fig, ax = plt.subplots(figsize=(7, 5))
for G, label, marker in [
    (G_fb, "Facebook (real)", "o"),
    (G_ba_fb, f"BA (n={n_fb}, m={m_ba})", "s"),
]:
    degrees = [d for _, d in G.degree()]
    deg_count = Counter(degrees)
    x, y = zip(*sorted(deg_count.items()))
    ax.scatter(
        x,
        y,
        s=30,
        alpha=0.7,
        marker=marker,
        label=label,
        edgecolors="white",
        linewidth=0.3,
    )
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Degree (k)")
ax.set_ylabel("Count")
ax.set_title("Facebook vs BA Model")
ax.legend()
fig.tight_layout()
plt.show()

# Compare stats
print(f"{'':20s} {'Facebook':>10s} {'BA model':>10s}")
print(
    f"{'Avg clustering':20s} {nx.average_clustering(G_fb):10.4f} {nx.average_clustering(G_ba_fb):10.4f}"
)
print(
    f"{'Max degree':20s} {max(d for _, d in G_fb.degree()):10d} {max(d for _, d in G_ba_fb.degree()):10d}"
)
print(f"\nBA captures hubs but misses the high clustering of real social networks.")

**The gap**: BA captures the fat tail (hubs) remarkably well, but its clustering coefficient is an order of magnitude lower than Facebook’s. Real social networks have both hubs *and* tight friend-groups — BA explains the former but not the latter. Models that combine preferential attachment with local clustering (like the Holme-Kim model) can close this gap, but no single simple model captures everything.

---
## 7. Tweak & Observe

**Predict before you tweak**: Increasing *m* (edges per new node) in BA means each newcomer makes more connections. Will this increase or decrease the maximum hub degree? Will it change the *shape* of the distribution or just shift it?

In [ ]:
# ---- TWEAK: Change m in BA, observe how hub size changes ----
m_tweak = 3  # <-- change me (try 1, 3, 5, 10)

G_ba_tw = models.barabasi_albert(n=500, m=m_tweak)
max_deg = max(d for _, d in G_ba_tw.degree())
print(f"BA(n=500, m={m_tweak}): max degree = {max_deg}")
viz.plot_degree_dist(G_ba_tw, log=True, title=f"BA (m={m_tweak}) — log-log")

**What you should see**: Increasing *m* raises the average degree and shifts the entire distribution rightward, but the **power-law slope stays roughly the same** (γ ≈ 3). The maximum hub degree increases because more connections are available overall, but the relative inequality between hubs and leaves persists. The shape is governed by the preferential attachment mechanism, not by *m* alone.

---
## 8. The Right Way to Visualize Power Laws

The log-log scatter plots we've been using have a problem: **noisy tails**. At high degree values, there are very few nodes, so the points scatter wildly. Binning into histograms introduces artifacts (the result depends heavily on bin width).

The solution: the **Complementary CDF (CCDF)**, which plots P(K ≥ k) — the probability that a randomly chosen node has degree *at least* k. On log-log axes:
- A **power law** → straight line with slope -(γ-1)
- An **exponential** → downward-curving line
- A **Poisson** → steep drop-off

The CCDF is cumulative and requires no binning, giving a much cleaner signal.

In [ ]:
# CCDF comparison: Facebook vs BA vs ER
fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))
graphs = [
    ("Facebook (real)", G_fb),
    ("BA model (n=500)", G_ba),
    ("ER model (n=500)", G_er),
]

for ax, (name, G) in zip(axes, graphs):
    degrees = sorted([d for _, d in G.degree() if d > 0], reverse=True)
    n = len(degrees)
    ccdf_y = np.arange(1, n + 1) / n
    ax.scatter(degrees, ccdf_y, s=15, alpha=0.7, edgecolors="white", linewidth=0.3)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Degree (k)")
    ax.set_ylabel("P(K ≥ k)")
    ax.set_title(name)

fig.suptitle(
    "CCDF: Power Law → Straight Line, Exponential → Curved",
    fontsize=13,
    fontweight="bold",
)
fig.tight_layout()
plt.show()

**Reading the CCDF**: Facebook and BA both show approximately straight lines on the CCDF — the hallmark of a power-law distribution. ER curves downward sharply, confirming its exponential (Poisson) tail. The CCDF gives a much cleaner picture than the raw degree count scatter because every data point contributes to the curve without binning artifacts.

In [ ]:
# Use the viz helper for a quick CCDF with fit line
viz.plot_ccdf(G_fb, title="Facebook CCDF with MLE fit", fit_line=True)

---
## 9. Power-Law Fitting (MLE)

How do we estimate the exponent γ of a power law P(k) ~ k^(-γ)?

**Maximum Likelihood Estimation (MLE)** gives the best estimate:

$$\hat{\alpha} = 1 + n \left[ \sum_{i=1}^{n} \ln \frac{x_i}{x_{\min}} \right]^{-1}$$

where x_i are the observed degree values ≥ x_min, and n is the count of such values.

The intuition: MLE finds the exponent that makes the observed data most probable under the power-law model. Unlike fitting a line to a log-log plot (which gives biased estimates), MLE is statistically principled.

In [ ]:
# Apply to Facebook, airports, and BA model
networks = [
    ("Facebook", [d for _, d in G_fb.degree()]),
    ("Airports", [d for _, d in G_air.degree()]),
    ("BA model", [d for _, d in G_ba.degree()]),
]

for name, degs in networks:
    alpha = fit_power_law(degs, k_min=2)
    print(f"{name:12s}: α = {alpha:.2f}  (γ = α ≈ {alpha:.1f})")

print(f"\nNote: BA theory predicts γ = 3.0 exactly.")
print(f"Real networks typically have γ between 2 and 3.")

In [ ]:
# Show MLE fit line on CCDF for all three
fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

for ax, (name, degs) in zip(axes, networks):
    degs_sorted = sorted([d for d in degs if d > 0], reverse=True)
    n = len(degs_sorted)
    ccdf_y = np.arange(1, n + 1) / n
    ax.scatter(degs_sorted, ccdf_y, s=15, alpha=0.6, edgecolors="white", linewidth=0.3)

    # MLE fit line
    k_min = 2
    alpha = fit_power_law(degs, k_min=k_min)
    k_range = np.logspace(np.log10(k_min), np.log10(max(degs)), 200)
    fit_y = (k_range / k_min) ** (-(alpha - 1))
    ax.plot(k_range, fit_y, "r--", lw=1.5, label=f"α = {alpha:.2f}")

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Degree (k)")
    ax.set_ylabel("P(K ≥ k)")
    ax.set_title(name)
    ax.legend(fontsize=9)

fig.suptitle("CCDF with MLE Power-Law Fit", fontsize=13, fontweight="bold")
fig.tight_layout()
plt.show()

**Interpreting the exponents**: The BA model's α ≈ 3.0 matches the theoretical prediction exactly — this is a strong validation of the MLE method. Real networks (Facebook, airports) deviate from γ = 3 because they involve more complex mechanisms than pure preferential attachment.

---
## 10. Is It Really a Power Law?

Many networks *look* like power laws on a log-log plot, but are they really? Let's compare the Facebook degree distribution against three theoretical distributions:

1. **Power law**: P(k) ~ k^(-γ) — straight line on CCDF
2. **Exponential**: P(k) ~ e^(-k/λ) — the ER/Poisson tail
3. **Log-normal**: P(k) ~ (1/k) · e^(-(ln k - μ)²/2σ²) — often confused with power laws

The key message: **proper methodology matters**. Clauset, Shalizi & Newman (2009) showed that many "scale-free" claims in the literature don't survive rigorous statistical testing.

In [ ]:
# Compare Facebook CCDF to theoretical distributions
fb_degs = sorted([d for _, d in G_fb.degree() if d > 0], reverse=True)
n = len(fb_degs)
ccdf_y = np.arange(1, n + 1) / n

# Theoretical CCDFs
k_range = np.logspace(np.log10(1), np.log10(max(fb_degs)), 300)

# Power law fit
alpha_fb = fit_power_law([d for _, d in G_fb.degree()], k_min=2)
ccdf_pl = (k_range / 2) ** (-(alpha_fb - 1))
ccdf_pl = ccdf_pl / ccdf_pl[0]  # normalize

# Exponential fit (match mean)
mean_deg = np.mean([d for _, d in G_fb.degree()])
ccdf_exp = np.exp(-k_range / mean_deg)
ccdf_exp = ccdf_exp / ccdf_exp[0]

# Poisson (what ER produces)
ccdf_poisson = 1 - poisson.cdf(k_range, mean_deg)
ccdf_poisson = np.maximum(ccdf_poisson, 1e-10)  # avoid log(0)

with plt.style.context("seaborn-v0_8-muted"):
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.scatter(
        fb_degs,
        ccdf_y,
        s=15,
        alpha=0.5,
        label="Facebook (data)",
        zorder=5,
        edgecolors="white",
        linewidth=0.3,
    )
    ax.plot(k_range, ccdf_pl, "r--", lw=1.5, label=f"Power law (\u03b1={alpha_fb:.2f})")
    ax.plot(k_range, ccdf_exp, "g-.", lw=1.5, label="Exponential")
    ax.plot(k_range, ccdf_poisson, "m:", lw=1.5, label=f"Poisson (\u03bb={mean_deg:.1f})")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Degree (k)")
    ax.set_ylabel("P(K \u2265 k)")
    ax.set_title("Facebook: Data vs Theoretical Distributions")
    ax.legend(fontsize=9)
    ax.set_ylim(bottom=1e-3)
    fig.tight_layout()
    plt.show()

**What the comparison shows**: The power-law fit tracks the data's tail much better than the exponential or Poisson. However, the fit isn't perfect — the data deviates at both low and high degrees. This is typical: real degree distributions are often *approximately* power-law in the tail but not across the full range. The debate over whether networks are "truly" scale-free remains active in the literature (Broido & Clauset, 2019).

**Takeaway**: Always use CCDF + MLE when claiming power-law behavior. Never rely on "it looks straight on a log-log plot."

---
## 11. Network Robustness

Hubs are powerful — but they're also Achilles' heels. What happens when they fail?

**Key concept**: removing nodes = site percolation. The central question is: at what fraction of removed nodes does the **giant component** vanish?

**The Molloy-Reed criterion** gives a condition for the giant component to exist:

$$\frac{\langle k^2 \rangle}{\langle k \rangle} > 2$$

where ⟨k⟩ is the average degree and ⟨k²⟩ is the average squared degree. When this ratio drops below 2, the giant component disintegrates.

In [ ]:
# Compute Molloy-Reed criterion for our networks
for name, G in [("Airports", G_air), ("Facebook", G_fb), ("Power Grid", G_pg)]:
    degrees = [d for _, d in G.degree()]
    k_avg = np.mean(degrees)
    k2_avg = np.mean(np.array(degrees) ** 2)
    kappa = k2_avg / k_avg
    print(f"{name:12s}: ⟨k⟩ = {k_avg:.1f},  ⟨k²⟩/⟨k⟩ = {kappa:.1f}  (threshold = 2)")
    print(f"{'':<12s}  Giant component {'EXISTS' if kappa > 2 else 'DOES NOT EXIST'}")
    print()

print("The higher ⟨k²⟩/⟨k⟩ is above 2, the more resilient the network.")
print(
    "Fat-tailed networks (airports) have very high ⟨k²⟩, making them robust to random failure."
)

**Predict before you run**: If we remove nodes randomly vs removing the highest-degree nodes first, which strategy will destroy the giant component faster? Think about what happens to ⟨k²⟩/⟨k⟩ in each case.

In [ ]:
# Random vs targeted attack sweep — track giant component size
fractions = np.arange(0, 0.51, 0.02)
rng_robust = np.random.default_rng(SEED)

results_robust = {}
for name, G in [("Airports", G_air), ("Facebook", G_fb)]:
    gcc_random = []
    gcc_targeted = []
    nodes = list(G.nodes())
    N = len(nodes)

    for f in fractions:
        n_remove = int(N * f)

        # Random removal
        G_rand = G.copy()
        remove_rand = set(rng_robust.choice(nodes, size=n_remove, replace=False))
        G_rand.remove_nodes_from(remove_rand)
        if G_rand.number_of_nodes() > 0:
            gcc_rand = len(max(nx.connected_components(G_rand), key=len)) / N
        else:
            gcc_rand = 0
        gcc_random.append(gcc_rand)

        # Targeted removal (highest degree first, recalculated)
        G_targ = G.copy()
        for _ in range(n_remove):
            if G_targ.number_of_nodes() == 0:
                break
            top_node = max(G_targ.nodes(), key=lambda n: G_targ.degree(n))
            G_targ.remove_node(top_node)
        if G_targ.number_of_nodes() > 0:
            gcc_targ = len(max(nx.connected_components(G_targ), key=len)) / N
        else:
            gcc_targ = 0
        gcc_targeted.append(gcc_targ)

    results_robust[name] = (gcc_random, gcc_targeted)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for ax, name in zip(axes, ["Airports", "Facebook"]):
    gcc_rand, gcc_targ = results_robust[name]
    ax.plot(fractions * 100, gcc_rand, "o-", label="Random failure", markersize=4)
    ax.plot(fractions * 100, gcc_targ, "s-", label="Targeted attack", markersize=4)
    ax.set_xlabel("% of nodes removed")
    ax.set_ylabel("Giant component (fraction of N)")
    ax.set_title(f"{name}")
    ax.legend(fontsize=9)
    ax.axhline(0, color="gray", linestyle=":", alpha=0.3)

fig.suptitle(
    "Network Robustness: Random Failure vs Targeted Attack",
    fontsize=13,
    fontweight="bold",
)
fig.tight_layout()
plt.show()

**The robustness paradox**: Scale-free networks (like airports) are remarkably **robust to random failure** — you can remove 30-40% of nodes randomly and the giant component barely shrinks. But they are **fragile to targeted attack** — removing just 10-15% of the highest-degree hubs shatters the network.

**The mathematical reason**: Random removal barely changes ⟨k²⟩/⟨k⟩ because most removed nodes are low-degree (the vast majority in a power-law distribution). But removing hubs destroys the high-k² terms that keep the ratio above 2.

This paradox has profound implications: the same hub structure that makes scale-free networks efficient also makes them vulnerable to deliberate attack.

In [ ]:
# Immunization concept: random removal vs targeted hub removal
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
pos_fb = nx.spring_layout(G_fb, seed=SEED)
rng_imm = np.random.default_rng(SEED)
frac_remove = 0.10
n_remove = int(G_fb.number_of_nodes() * frac_remove)

# Panel 1: Random removal
G_rand_rm = G_fb.copy()
rand_remove = set(rng_imm.choice(list(G_fb.nodes()), size=n_remove, replace=False))
G_rand_rm.remove_nodes_from(rand_remove)
comps_rand = sorted(nx.connected_components(G_rand_rm), key=len, reverse=True)
colors_rand = []
for n in G_fb.nodes():
    if n in rand_remove:
        colors_rand.append("#FFFFFF")
    elif n in comps_rand[0]:
        colors_rand.append("#4878CF")
    else:
        colors_rand.append("#CCCCCC")
sizes_rand = [0 if n in rand_remove else 30 for n in G_fb.nodes()]
nx.draw_networkx_nodes(
    G_fb, pos_fb, ax=axes[0], node_color=colors_rand, node_size=sizes_rand
)
edges_rand = [
    (u, v) for u, v in G_fb.edges() if u not in rand_remove and v not in rand_remove
]
nx.draw_networkx_edges(
    G_fb, pos_fb, ax=axes[0], edgelist=edges_rand, edge_color="#cccccc", width=0.3
)
gcc_rand = len(comps_rand[0]) / G_fb.number_of_nodes()
axes[0].set_title(
    f"Random removal ({frac_remove:.0%} of nodes)\nLargest component: {gcc_rand:.0%} of network",
    fontsize=11,
)
axes[0].axis("off")

# Panel 2: Targeted removal (top hubs)
G_targ_rm = G_fb.copy()
hubs_sorted = sorted(G_fb.nodes(), key=lambda n: G_fb.degree(n), reverse=True)
targ_remove = set(hubs_sorted[:n_remove])
G_targ_rm.remove_nodes_from(targ_remove)
comps_targ = sorted(nx.connected_components(G_targ_rm), key=len, reverse=True)
colors_targ = []
for n in G_fb.nodes():
    if n in targ_remove:
        colors_targ.append("#FFFFFF")
    elif n in comps_targ[0]:
        colors_targ.append("#D65F5F")
    else:
        colors_targ.append("#CCCCCC")
sizes_targ = [0 if n in targ_remove else 30 for n in G_fb.nodes()]
nx.draw_networkx_nodes(
    G_fb, pos_fb, ax=axes[1], node_color=colors_targ, node_size=sizes_targ
)
edges_targ = [
    (u, v) for u, v in G_fb.edges() if u not in targ_remove and v not in targ_remove
]
nx.draw_networkx_edges(
    G_fb, pos_fb, ax=axes[1], edgelist=edges_targ, edge_color="#cccccc", width=0.3
)
gcc_targ = len(comps_targ[0]) / G_fb.number_of_nodes()
axes[1].set_title(
    f"Targeted hub removal ({frac_remove:.0%} of nodes)\nLargest component: {gcc_targ:.0%} of network",
    fontsize=11,
)
axes[1].axis("off")

fig.suptitle(
    "Robustness Paradox: Random vs Targeted Removal", fontsize=13, fontweight="bold"
)
fig.tight_layout()
plt.show()

**The hub removal effect**: Removing 10% of nodes at random (left) barely dents the network — the giant component remains mostly intact. But removing the top 10% highest-degree hubs (right) shatters it into disconnected fragments. This asymmetry is why targeted attacks on infrastructure networks (power grids, internet backbone) are so dangerous — and why understanding hubs matters for network defense.

---
## Summary

| Model | Degree dist | Clustering | Hubs? | Key mechanism |
|-------|------------|-----------|-------|---------------|
| **Erdos-Renyi** | Poisson (narrow) | Low | No | Random edges |
| **Watts-Strogatz** | Narrow (near-uniform) | High | No | Rewired lattice |
| **Barabasi-Albert** | Power law (fat tail) | Low | Yes | Preferential attachment |

### Analysis Methods

| Method | Purpose | Key insight |
|--------|---------|-------------|
| **CCDF plot** | Visualize degree distributions cleanly | No binning artifacts; power law = straight line |
| **MLE fitting** | Estimate power-law exponent | Statistically principled; avoids log-log regression bias |
| **Distribution comparison** | Test "is it really a power law?" | Compare power law vs exponential vs Poisson |
| **Molloy-Reed criterion** | Test giant component existence | ⟨k²⟩/⟨k⟩ > 2 for connected network |
| **Robustness paradox** | Targeted vs random failure | Hubs = robust to random, fragile to targeted |

No single model captures everything about real networks. BA explains hubs but misses clustering.
WS explains clustering but misses hubs. And the hubs that make networks efficient also make them vulnerable to targeted attack.

Next week: **Community Detection** — finding groups within networks.